In [3]:
# Install Python for S3 File System if necessary: pip install s3fs
from kafka import KafkaConsumer
from time import sleep
from json import dumps,loads
import json
from s3fs import S3FileSystem
from datetime import datetime

In [4]:
# Create the consumer object on the EC2 public IP where Kafka is running
consumer = KafkaConsumer('test1',
                         bootstrap_servers=['3.20.222.206:9092'],
                         value_deserializer=lambda x: loads(x.decode('utf-8'))) # UTF-8 decoding to JSON format

In [ ]:
for c in consumer:
    print(c.value) # Testing: print data in real-time as Producer creates it

In [ ]:
from s3fs import S3FileSystem 

s3 = S3FileSystem() # create the S3FS object
bucket_path = "s3://kafka-stock-streaming-project"

def save_to_s3(content, filename):
    with s3.open(f'{bucket_path}/{filename}', 'w') as file:
        json.dump(content, file) # Dump data that consumer receives into S3 in real-time

for message in consumer:
    try:
        data = json.loads(message.value)
        stock_symbol = data['ticker']
        stock_data[stock_symbol] = data

        # Check if we have data for all stocks
        if all(symbol in stock_data for symbol in ['AAPL', 'GOOGL', 'TSLA', 'MSFT', 'AMZN']):
            filename = f"stock_data_{datetime.now().strftime('%Y%m%d%H%M%S')}.json"
            save_to_s3(stock_data, filename)
            stock_data = {}  # Reset for next group

    except Exception as e:
        print(f"Error processing {message}: {e}")

consumer.close()